In [3]:
#!/usr/bin/env python3
"""
Multi-omics Differential Features Analysis Script
Analyzes DEGs, DEmiRs, and DMRs distribution and characteristics
Author: Analysis Pipeline
Date: 2025
"""

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import warnings
import os
from pathlib import Path

# Suppress warnings for cleaner output
warnings.filterwarnings('ignore')

# Set plotting style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

# Configuration
INPUT_DIR = "/Users/heweilin/Desktop/P056"
OUTPUT_DIR = "/Users/heweilin/Desktop/P056_Code_3/Figure"
DPI = 300
FIGSIZE = (12, 8)

# Create output directory if it doesn't exist
os.makedirs(OUTPUT_DIR, exist_ok=True)

def load_and_process_data():
    """
    Load and preprocess the differential expression data
    Returns processed dataframes for DEGs, DEmiRs, and DMRs
    """
    print("Loading differential expression data...")
    
    # Load DEGs (mRNA)
    degs_file = os.path.join(INPUT_DIR, "1mRNA_DEGs_proteincoding.csv")
    degs_df = pd.read_csv(degs_file)
    print(f"Loaded {len(degs_df)} mRNA records")
    
    # Load DEmiRs
    demirs_file = os.path.join(INPUT_DIR, "2miRNA_DEmirs.csv")
    demirs_df = pd.read_csv(demirs_file)
    print(f"Loaded {len(demirs_df)} miRNA records")
    
    # Load DMRs
    dmrs_file = os.path.join(INPUT_DIR, "4DNA_DMRs.csv")
    dmrs_df = pd.read_csv(dmrs_file)
    print(f"Loaded {len(dmrs_df)} DMR records")
    
    return degs_df, demirs_df, dmrs_df

def classify_differential_features(degs_df, demirs_df, dmrs_df):
    """
    Classify features as significantly differential or not
    Returns classification results
    """
    print("\nClassifying differential features...")
    
    # Classify DEGs (using padj < 0.05)
    degs_significant = degs_df[degs_df['padj'] < 0.05]
    degs_upregulated = degs_significant[degs_significant['log2FoldChange'] > 0]
    degs_downregulated = degs_significant[degs_significant['log2FoldChange'] < 0]
    
    print(f"DEGs: {len(degs_significant)} significant out of {len(degs_df)} total")
    print(f"  - Upregulated: {len(degs_upregulated)}")
    print(f"  - Downregulated: {len(degs_downregulated)}")
    
    # Classify DEmiRs (using pvalue < 0.05 as mentioned in readme)
    demirs_significant = demirs_df[demirs_df['pvalue'] < 0.05]
    demirs_upregulated = demirs_significant[demirs_significant['log2FoldChange'] > 0]
    demirs_downregulated = demirs_significant[demirs_significant['log2FoldChange'] < 0]
    
    print(f"DEmiRs: {len(demirs_significant)} significant out of {len(demirs_df)} total")
    print(f"  - Upregulated: {len(demirs_upregulated)}")
    print(f"  - Downregulated: {len(demirs_downregulated)}")
    
    # DMRs are already filtered (significant only)
    dmrs_hyper = dmrs_df[dmrs_df['DiffMethylated'] == 'HYPER']
    dmrs_hypo = dmrs_df[dmrs_df['DiffMethylated'] == 'HYPO']
    
    print(f"DMRs: {len(dmrs_df)} significant regions")
    print(f"  - Hypermethylated: {len(dmrs_hyper)}")
    print(f"  - Hypomethylated: {len(dmrs_hypo)}")
    
    # Create summary dictionary
    summary = {
        'DEGs': {
            'total': len(degs_df),
            'significant': len(degs_significant),
            'upregulated': len(degs_upregulated),
            'downregulated': len(degs_downregulated)
        },
        'DEmiRs': {
            'total': len(demirs_df),
            'significant': len(demirs_significant),
            'upregulated': len(demirs_upregulated),
            'downregulated': len(demirs_downregulated)
        },
        'DMRs': {
            'total': len(dmrs_df),
            'significant': len(dmrs_df),
            'hypermethylated': len(dmrs_hyper),
            'hypomethylated': len(dmrs_hypo)
        }
    }
    
    return summary, degs_significant, demirs_significant, dmrs_df

def create_overview_barplot(summary):
    """
    Create overview barplot showing total and significant features
    """
    print("\nCreating overview barplot...")
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
    
    # Total vs Significant counts
    categories = ['DEGs', 'DEmiRs', 'DMRs']
    total_counts = [summary[cat]['total'] for cat in categories]
    significant_counts = [summary[cat]['significant'] for cat in categories]
    
    x_pos = np.arange(len(categories))
    width = 0.35
    
    bars1 = ax1.bar(x_pos - width/2, total_counts, width, label='Total', alpha=0.7)
    bars2 = ax1.bar(x_pos + width/2, significant_counts, width, label='Significant', alpha=0.7)
    
    ax1.set_xlabel('Feature Type')
    ax1.set_ylabel('Count')
    ax1.set_title('Total vs Significant Differential Features')
    ax1.set_xticks(x_pos)
    ax1.set_xticklabels(categories)
    ax1.legend()
    ax1.grid(axis='y', alpha=0.3)
    
    # Add value labels on bars
    for bar in bars1:
        height = bar.get_height()
        ax1.text(bar.get_x() + bar.get_width()/2., height + height*0.01,
                f'{int(height)}', ha='center', va='bottom', fontsize=10)
    
    for bar in bars2:
        height = bar.get_height()
        ax1.text(bar.get_x() + bar.get_width()/2., height + height*0.01,
                f'{int(height)}', ha='center', va='bottom', fontsize=10)
    
    # Direction-specific counts (Up/Down regulated)
    deg_counts = [summary['DEGs']['upregulated'], summary['DEGs']['downregulated']]
    demir_counts = [summary['DEmiRs']['upregulated'], summary['DEmiRs']['downregulated']]
    dmr_counts = [summary['DMRs']['hypermethylated'], summary['DMRs']['hypomethylated']]
    
    x_pos2 = np.arange(3)
    width2 = 0.25
    
    bars3 = ax2.bar(x_pos2 - width2, [deg_counts[0], demir_counts[0], dmr_counts[0]], 
                    width2, label='Up/Hyper', alpha=0.7, color='red')
    bars4 = ax2.bar(x_pos2 + width2, [deg_counts[1], demir_counts[1], dmr_counts[1]], 
                    width2, label='Down/Hypo', alpha=0.7, color='blue')
    
    ax2.set_xlabel('Feature Type')
    ax2.set_ylabel('Count')
    ax2.set_title('Direction of Differential Features')
    ax2.set_xticks(x_pos2)
    ax2.set_xticklabels(['DEGs', 'DEmiRs', 'DMRs'])
    ax2.legend()
    ax2.grid(axis='y', alpha=0.3)
    
    # Add value labels
    for bars in [bars3, bars4]:
        for bar in bars:
            height = bar.get_height()
            ax2.text(bar.get_x() + bar.get_width()/2., height + height*0.01,
                    f'{int(height)}', ha='center', va='bottom', fontsize=10)
    
    plt.tight_layout()
    plt.savefig(os.path.join(OUTPUT_DIR, "01_differential_features_overview.png"), 
                dpi=DPI, bbox_inches='tight')
    plt.close()

def create_volcano_plots(degs_df, demirs_df):
    """
    Create volcano plots for DEGs and DEmiRs
    """
    print("\nCreating volcano plots...")
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))
    
    # DEGs volcano plot
    degs_df['-log10(padj)'] = -np.log10(degs_df['padj'].replace(0, np.finfo(float).tiny))
    
    # Color points based on significance and fold change
    colors = []
    for _, row in degs_df.iterrows():
        if row['padj'] < 0.05 and abs(row['log2FoldChange']) > 1:
            if row['log2FoldChange'] > 1:
                colors.append('red')
            else:
                colors.append('blue')
        else:
            colors.append('grey')
    
    scatter1 = ax1.scatter(degs_df['log2FoldChange'], degs_df['-log10(padj)'], 
                          c=colors, alpha=0.6, s=20)
    
    # Add significance lines
    ax1.axhline(y=-np.log10(0.05), color='black', linestyle='--', alpha=0.5)
    ax1.axvline(x=1, color='black', linestyle='--', alpha=0.5)
    ax1.axvline(x=-1, color='black', linestyle='--', alpha=0.5)
    
    ax1.set_xlabel('Log2 Fold Change')
    ax1.set_ylabel('-Log10(adjusted p-value)')
    ax1.set_title(f'DEGs Volcano Plot (n={len(degs_df)})')
    ax1.grid(True, alpha=0.3)
    
    # DEmiRs volcano plot
    demirs_df['-log10(pvalue)'] = -np.log10(demirs_df['pvalue'].replace(0, np.finfo(float).tiny))
    
    # Color points for miRNAs
    colors2 = []
    for _, row in demirs_df.iterrows():
        if row['pvalue'] < 0.05 and abs(row['log2FoldChange']) > 1:
            if row['log2FoldChange'] > 1:
                colors2.append('red')
            else:
                colors2.append('blue')
        else:
            colors2.append('grey')
    
    scatter2 = ax2.scatter(demirs_df['log2FoldChange'], demirs_df['-log10(pvalue)'], 
                          c=colors2, alpha=0.6, s=20)
    
    # Add significance lines
    ax2.axhline(y=-np.log10(0.05), color='black', linestyle='--', alpha=0.5)
    ax2.axvline(x=1, color='black', linestyle='--', alpha=0.5)
    ax2.axvline(x=-1, color='black', linestyle='--', alpha=0.5)
    
    ax2.set_xlabel('Log2 Fold Change')
    ax2.set_ylabel('-Log10(p-value)')
    ax2.set_title(f'DEmiRs Volcano Plot (n={len(demirs_df)})')
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig(os.path.join(OUTPUT_DIR, "02_volcano_plots.png"), 
                dpi=DPI, bbox_inches='tight')
    plt.close()

def analyze_chromosomal_distribution(degs_significant, dmrs_df):
    """
    Analyze chromosomal distribution of DEGs and DMRs
    """
    print("\nAnalyzing chromosomal distribution...")
    
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(15, 10))
    
    # DEGs chromosomal distribution
    deg_chr_counts = degs_significant['Chromosome'].value_counts().sort_index()
    
    # Sort chromosomes properly (1, 2, ..., 22, X, Y)
    chr_order = [str(i) for i in range(1, 23)] + ['X', 'Y', 'MT']
    deg_chr_counts = deg_chr_counts.reindex([c for c in chr_order if c in deg_chr_counts.index])
    
    bars1 = ax1.bar(range(len(deg_chr_counts)), deg_chr_counts.values, alpha=0.7)
    ax1.set_xlabel('Chromosome')
    ax1.set_ylabel('Number of DEGs')
    ax1.set_title(f'Chromosomal Distribution of DEGs (n={len(degs_significant)})')
    ax1.set_xticks(range(len(deg_chr_counts)))
    ax1.set_xticklabels(deg_chr_counts.index, rotation=45)
    ax1.grid(axis='y', alpha=0.3)
    
    # Add value labels on bars
    for i, bar in enumerate(bars1):
        height = bar.get_height()
        ax1.text(bar.get_x() + bar.get_width()/2., height + height*0.01,
                f'{int(height)}', ha='center', va='bottom', fontsize=8, rotation=90)
    
    # DMRs chromosomal distribution
    dmrs_df['chromosome'] = dmrs_df['seqnames'].str.replace('chr', '')
    dmr_chr_counts = dmrs_df['chromosome'].value_counts().sort_index()
    dmr_chr_counts = dmr_chr_counts.reindex([c for c in chr_order if c in dmr_chr_counts.index])
    
    bars2 = ax2.bar(range(len(dmr_chr_counts)), dmr_chr_counts.values, alpha=0.7, color='orange')
    ax2.set_xlabel('Chromosome')
    ax2.set_ylabel('Number of DMRs')
    ax2.set_title(f'Chromosomal Distribution of DMRs (n={len(dmrs_df)})')
    ax2.set_xticks(range(len(dmr_chr_counts)))
    ax2.set_xticklabels(dmr_chr_counts.index, rotation=45)
    ax2.grid(axis='y', alpha=0.3)
    
    # Add value labels on bars
    for i, bar in enumerate(bars2):
        height = bar.get_height()
        ax2.text(bar.get_x() + bar.get_width()/2., height + height*0.01,
                f'{int(height)}', ha='center', va='bottom', fontsize=8, rotation=90)
    
    plt.tight_layout()
    plt.savefig(os.path.join(OUTPUT_DIR, "03_chromosomal_distribution.png"), 
                dpi=DPI, bbox_inches='tight')
    plt.close()
    
    return deg_chr_counts, dmr_chr_counts

def analyze_fold_change_distributions(degs_significant, demirs_significant):
    """
    Analyze fold change distributions
    """
    print("\nAnalyzing fold change distributions...")
    
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))
    
    # DEGs fold change histogram
    axes[0,0].hist(degs_significant['log2FoldChange'], bins=30, alpha=0.7, color='skyblue', edgecolor='black')
    axes[0,0].set_xlabel('Log2 Fold Change')
    axes[0,0].set_ylabel('Frequency')
    axes[0,0].set_title(f'DEGs Log2 Fold Change Distribution (n={len(degs_significant)})')
    axes[0,0].axvline(x=0, color='red', linestyle='--', alpha=0.7)
    axes[0,0].grid(axis='y', alpha=0.3)
    
    # DEmiRs fold change histogram
    axes[0,1].hist(demirs_significant['log2FoldChange'], bins=20, alpha=0.7, color='lightcoral', edgecolor='black')
    axes[0,1].set_xlabel('Log2 Fold Change')
    axes[0,1].set_ylabel('Frequency')
    axes[0,1].set_title(f'DEmiRs Log2 Fold Change Distribution (n={len(demirs_significant)})')
    axes[0,1].axvline(x=0, color='red', linestyle='--', alpha=0.7)
    axes[0,1].grid(axis='y', alpha=0.3)
    
    # Box plots for comparison
    data_to_plot = [degs_significant['log2FoldChange'], demirs_significant['log2FoldChange']]
    bp1 = axes[1,0].boxplot(data_to_plot, labels=['DEGs', 'DEmiRs'], patch_artist=True)
    bp1['boxes'][0].set_facecolor('skyblue')
    bp1['boxes'][1].set_facecolor('lightcoral')
    axes[1,0].set_ylabel('Log2 Fold Change')
    axes[1,0].set_title('Fold Change Comparison')
    axes[1,0].grid(axis='y', alpha=0.3)
    axes[1,0].axhline(y=0, color='red', linestyle='--', alpha=0.7)
    
    # Statistical summary
    deg_stats = degs_significant['log2FoldChange'].describe()
    demir_stats = demirs_significant['log2FoldChange'].describe()
    
    stats_text = f"""DEGs Statistics:
Mean: {deg_stats['mean']:.3f}
Median: {deg_stats['50%']:.3f}
Std: {deg_stats['std']:.3f}
Min: {deg_stats['min']:.3f}
Max: {deg_stats['max']:.3f}

DEmiRs Statistics:
Mean: {demir_stats['mean']:.3f}
Median: {demir_stats['50%']:.3f}
Std: {demir_stats['std']:.3f}
Min: {demir_stats['min']:.3f}
Max: {demir_stats['max']:.3f}"""
    
    axes[1,1].text(0.05, 0.95, stats_text, transform=axes[1,1].transAxes, 
                   verticalalignment='top', fontfamily='monospace', fontsize=10)
    axes[1,1].set_title('Statistical Summary')
    axes[1,1].axis('off')
    
    plt.tight_layout()
    plt.savefig(os.path.join(OUTPUT_DIR, "04_fold_change_analysis.png"), 
                dpi=DPI, bbox_inches='tight')
    plt.close()

def analyze_dmr_characteristics(dmrs_df):
    """
    Analyze DMR characteristics
    """
    print("\nAnalyzing DMR characteristics...")
    
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))
    
    # DMR width distribution
    axes[0,0].hist(dmrs_df['width'], bins=30, alpha=0.7, color='green', edgecolor='black')
    axes[0,0].set_xlabel('DMR Width (bp)')
    axes[0,0].set_ylabel('Frequency')
    axes[0,0].set_title(f'DMR Width Distribution (n={len(dmrs_df)})')
    axes[0,0].grid(axis='y', alpha=0.3)
    
    # DMR type distribution (exon vs intron vs other)
    dmr_type_counts = dmrs_df['annot.type'].value_counts()
    axes[0,1].pie(dmr_type_counts.values, labels=dmr_type_counts.index, autopct='%1.1f%%', startangle=90)
    axes[0,1].set_title('DMR Genomic Region Distribution')
    
    # Hyper vs Hypo methylation
    meth_status_counts = dmrs_df['DiffMethylated'].value_counts()
    bars = axes[1,0].bar(meth_status_counts.index, meth_status_counts.values, 
                        color=['red', 'blue'], alpha=0.7)
    axes[1,0].set_xlabel('Methylation Status')
    axes[1,0].set_ylabel('Count')
    axes[1,0].set_title('DMR Methylation Status Distribution')
    axes[1,0].grid(axis='y', alpha=0.3)
    
    # Add value labels on bars
    for bar in bars:
        height = bar.get_height()
        axes[1,0].text(bar.get_x() + bar.get_width()/2., height + height*0.01,
                      f'{int(height)}', ha='center', va='bottom', fontsize=12)
    
    # P-value vs Q-value scatter
    scatter = axes[1,1].scatter(dmrs_df['pvalue'], dmrs_df['qvalue'], 
                               c=dmrs_df['DiffMethylated'].map({'HYPER': 'red', 'HYPO': 'blue'}),
                               alpha=0.6, s=20)
    axes[1,1].set_xlabel('P-value')
    axes[1,1].set_ylabel('Q-value')
    axes[1,1].set_title('DMR P-value vs Q-value')
    axes[1,1].set_xscale('log')
    axes[1,1].set_yscale('log')
    axes[1,1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig(os.path.join(OUTPUT_DIR, "05_dmr_characteristics.png"), 
                dpi=DPI, bbox_inches='tight')
    plt.close()

def create_summary_heatmap(summary):
    """
    Create a summary heatmap showing all differential features
    """
    print("\nCreating summary heatmap...")
    
    # Prepare data for heatmap
    heatmap_data = pd.DataFrame({
        'DEGs': [summary['DEGs']['upregulated'], summary['DEGs']['downregulated']],
        'DEmiRs': [summary['DEmiRs']['upregulated'], summary['DEmiRs']['downregulated']],
        'DMRs': [summary['DMRs']['hypermethylated'], summary['DMRs']['hypomethylated']]
    }, index=['Up/Hyper', 'Down/Hypo'])
    
    fig, ax = plt.subplots(figsize=(8, 6))
    
    # Create heatmap
    sns.heatmap(heatmap_data, annot=True, fmt='d', cmap='RdYlBu_r', 
                cbar_kws={'label': 'Count'}, ax=ax)
    
    ax.set_title('Differential Features Summary Heatmap')
    ax.set_xlabel('Feature Type')
    ax.set_ylabel('Direction')
    
    plt.tight_layout()
    plt.savefig(os.path.join(OUTPUT_DIR, "06_summary_heatmap.png"), 
                dpi=DPI, bbox_inches='tight')
    plt.close()

def generate_summary_report(summary, deg_chr_counts, dmr_chr_counts):
    """
    Generate a comprehensive summary report
    """
    print("\nGenerating summary report...")
    
    report = f"""
=================================================================
        MULTI-OMICS DIFFERENTIAL FEATURES ANALYSIS REPORT
=================================================================

OVERVIEW:
---------
This analysis identified differential features across three omics layers:
- Transcriptomics (mRNA): Differentially Expressed Genes (DEGs)
- Epigenomics (miRNA): Differentially Expressed miRNAs (DEmiRs)  
- Methylomics (DNA): Differentially Methylated Regions (DMRs)

DIFFERENTIAL FEATURES SUMMARY:
-----------------------------
DEGs (mRNA):
  - Total genes analyzed: {summary['DEGs']['total']:,}
  - Significantly differential: {summary['DEGs']['significant']:,} ({summary['DEGs']['significant']/summary['DEGs']['total']*100:.2f}%)
  - Upregulated: {summary['DEGs']['upregulated']:,}
  - Downregulated: {summary['DEGs']['downregulated']:,}

DEmiRs (miRNA):
  - Total miRNAs analyzed: {summary['DEmiRs']['total']:,}
  - Significantly differential: {summary['DEmiRs']['significant']:,} ({summary['DEmiRs']['significant']/summary['DEmiRs']['total']*100:.2f}%)
  - Upregulated: {summary['DEmiRs']['upregulated']:,}
  - Downregulated: {summary['DEmiRs']['downregulated']:,}

DMRs (DNA Methylation):
  - Total significant DMRs: {summary['DMRs']['total']:,}
  - Hypermethylated: {summary['DMRs']['hypermethylated']:,} ({summary['DMRs']['hypermethylated']/summary['DMRs']['total']*100:.2f}%)
  - Hypomethylated: {summary['DMRs']['hypomethylated']:,} ({summary['DMRs']['hypomethylated']/summary['DMRs']['total']*100:.2f}%)

CHROMOSOMAL DISTRIBUTION:
------------------------
Top 5 chromosomes with most DEGs:
"""
    
    for i, (chr_name, count) in enumerate(deg_chr_counts.head().items()):
        report += f"  {i+1}. Chromosome {chr_name}: {count} DEGs\n"
    
    report += f"\nTop 5 chromosomes with most DMRs:\n"
    for i, (chr_name, count) in enumerate(dmr_chr_counts.head().items()):
        report += f"  {i+1}. Chromosome {chr_name}: {count} DMRs\n"
    
    report += f"""
ANALYSIS OUTPUTS:
----------------
The following visualization files have been generated:
1. 01_differential_features_overview.png - Overview of total vs significant features
2. 02_volcano_plots.png - Volcano plots for DEGs and DEmiRs
3. 03_chromosomal_distribution.png - Chromosomal distribution of DEGs and DMRs
4. 04_fold_change_analysis.png - Fold change distribution analysis
5. 05_dmr_characteristics.png - DMR characteristics and distributions
6. 06_summary_heatmap.png - Summary heatmap of all differential features

All files saved to: {OUTPUT_DIR}

=================================================================
Analysis completed successfully!
Date: {pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S')}
=================================================================
"""
    
    print(report)
    
    # Save report to file
    with open(os.path.join(OUTPUT_DIR, "analysis_report.txt"), 'w') as f:
        f.write(report)

def main():
    """
    Main analysis pipeline
    """
    print("="*60)
    print("MULTI-OMICS DIFFERENTIAL FEATURES ANALYSIS")
    print("="*60)
    
    try:
        # Load data
        degs_df, demirs_df, dmrs_df = load_and_process_data()
        
        # Classify differential features
        summary, degs_significant, demirs_significant, dmrs_df = classify_differential_features(
            degs_df, demirs_df, dmrs_df)
        
        # Create visualizations
        create_overview_barplot(summary)
        create_volcano_plots(degs_df, demirs_df)
        deg_chr_counts, dmr_chr_counts = analyze_chromosomal_distribution(degs_significant, dmrs_df)
        analyze_fold_change_distributions(degs_significant, demirs_significant)
        analyze_dmr_characteristics(dmrs_df)
        create_summary_heatmap(summary)
        
        # Generate summary report
        generate_summary_report(summary, deg_chr_counts, dmr_chr_counts)
        
        print(f"\nAnalysis completed successfully!")
        print(f"All outputs saved to: {OUTPUT_DIR}")
        
    except Exception as e:
        print(f"Error occurred during analysis: {str(e)}")
        import traceback
        traceback.print_exc()

if __name__ == "__main__":
    main()

MULTI-OMICS DIFFERENTIAL FEATURES ANALYSIS
Loading differential expression data...
Loaded 19853 mRNA records
Loaded 2201 miRNA records
Loaded 493648 DMR records

Classifying differential features...
DEGs: 208 significant out of 19853 total
  - Upregulated: 138
  - Downregulated: 70
DEmiRs: 46 significant out of 2201 total
  - Upregulated: 30
  - Downregulated: 16
DMRs: 493648 significant regions
  - Hypermethylated: 297021
  - Hypomethylated: 196627

Creating overview barplot...

Creating volcano plots...

Analyzing chromosomal distribution...

Analyzing fold change distributions...

Analyzing DMR characteristics...

Creating summary heatmap...

Generating summary report...

        MULTI-OMICS DIFFERENTIAL FEATURES ANALYSIS REPORT

OVERVIEW:
---------
This analysis identified differential features across three omics layers:
- Transcriptomics (mRNA): Differentially Expressed Genes (DEGs)
- Epigenomics (miRNA): Differentially Expressed miRNAs (DEmiRs)  
- Methylomics (DNA): Differential